In [1]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
cd_train=pd.read_csv(r'C:\Users\satis\OneDrive\Desktop\Edvancer\Python\project_1\P1 Data\Consumer_Complaints_train.csv')
cd_test=pd.read_csv(r'C:\Users\satis\OneDrive\Desktop\Edvancer\Python\project_1\P1 Data\Consumer_Complaints_test_share.csv')

In [3]:
cd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
Date received                   478421 non-null object
Product                         478421 non-null object
Sub-product                     339948 non-null object
Issue                           478421 non-null object
Sub-issue                       185796 non-null object
Consumer complaint narrative    75094 non-null object
Company public response         90392 non-null object
Company                         478421 non-null object
State                           474582 non-null object
ZIP code                        474573 non-null object
Tags                            67206 non-null object
Consumer consent provided?      135487 non-null object
Submitted via                   478421 non-null object
Date sent to company            478421 non-null object
Company response to consumer    478421 non-null object
Timely response?                478421 non-null object
Consumer dis

In [4]:
feat_nan=[features for features in cd_train.columns if cd_train[features].isnull().sum()>1]

In [5]:
for feat in feat_nan:
    print(feat ,np.round(cd_train[feat].isnull().mean(),4),'   % missing value')

Sub-product 0.2894    % missing value
Sub-issue 0.6116    % missing value
Consumer complaint narrative 0.843    % missing value
Company public response 0.8111    % missing value
State 0.008    % missing value
ZIP code 0.008    % missing value
Tags 0.8595    % missing value
Consumer consent provided? 0.7168    % missing value


In [6]:
feat_nan

['Sub-product',
 'Sub-issue',
 'Consumer complaint narrative',
 'Company public response',
 'State',
 'ZIP code',
 'Tags',
 'Consumer consent provided?']

In [7]:
cd_train.isnull().sum()

Date received                        0
Product                              0
Sub-product                     138473
Issue                                0
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
Company                              0
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype: int64

In [8]:
cd_test.shape

(119606, 17)

In [9]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Date received : 1759
Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company public response : 10
Company : 3276
State : 62
ZIP code : 25962
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Date sent to company : 1706
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [10]:
for col in ['Date received','Date sent to company']:
    cd_train[col]=pd.to_datetime(cd_train[col],infer_datetime_format=True)
    cd_test[col]=pd.to_datetime(cd_test[col],infer_datetime_format=True)

In [11]:
cd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
Date received                   478421 non-null datetime64[ns]
Product                         478421 non-null object
Sub-product                     339948 non-null object
Issue                           478421 non-null object
Sub-issue                       185796 non-null object
Consumer complaint narrative    75094 non-null object
Company public response         90392 non-null object
Company                         478421 non-null object
State                           474582 non-null object
ZIP code                        474573 non-null object
Tags                            67206 non-null object
Consumer consent provided?      135487 non-null object
Submitted via                   478421 non-null object
Date sent to company            478421 non-null datetime64[ns]
Company response to consumer    478421 non-null object
Timely response?                478421 non-null obj

In [12]:
cd_train['day_diff']=pd.to_numeric(cd_train['Date sent to company']-cd_train['Date received'])
cd_test['day_diff']=pd.to_numeric(cd_test['Date sent to company']-cd_test['Date received'])

In [13]:
for col in ['Date received','Date sent to company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [14]:
for col in ['ZIP code','Company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

In [15]:
cd_test.head()

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,State,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Complaint ID,day_diff
0,Bank account or service,Cashing a check without an account,Deposits and withdrawals,NaN,NaN,NaN,CA,NaN,NaN,Web,Closed with explanation,Yes,675956,-86400000000000
1,Debt collection,Credit card,Cont'd attempts collect debt not owed,Debt was paid,NaN,NaN,FL,NaN,Consent not provided,Web,Closed with explanation,Yes,1858795,0
2,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,NaN,CA,NaN,NaN,Web,Closed without relief,Yes,32637,86400000000000
3,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,Company chooses not to provide a public response,FL,Older American,NaN,Postal mail,Closed with non-monetary relief,Yes,1731374,432000000000000
4,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,FL,NaN,NaN,Web,Closed with explanation,Yes,501487,0


In [16]:
cd_train['Consumer disputed?']=np.where(cd_train['Consumer disputed?']=="Yes",1,0)

In [17]:
cd_train.head(8)

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,State,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,day_diff
0,Credit card,NaN,Billing statement,NaN,NaN,NaN,MI,Older American,NaN,Web,Closed with explanation,Yes,0,856103,86400000000000
1,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,PA,NaN,NaN,Referral,Closed,Yes,0,1034666,518400000000000
2,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,CA,NaN,NaN,Referral,Closed with non-monetary relief,Yes,0,756363,1814400000000000
3,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,GA,Older American,Consent provided,Web,Closed with monetary relief,Yes,0,1474177,0
4,Credit card,NaN,Transaction issue,NaN,NaN,NaN,MA,NaN,NaN,Web,Closed with explanation,Yes,0,1132572,691200000000000
5,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,WI,NaN,NaN,Referral,Closed with explanation,Yes,0,913769,432000000000000
6,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,FL,NaN,NaN,Referral,Closed with explanation,Yes,0,163903,259200000000000
7,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",NaN,I have a mortgage with Quicken Loans. From XXX...,NaN,GA,NaN,Consent provided,Web,Closed with explanation,Yes,1,1362945,0


In [18]:
m=cd_train['Issue'].value_counts()

In [19]:
m.head(10)

Loan modification,collection,foreclosure    80302
Incorrect information on credit report      58527
Loan servicing, payments, escrow account    51403
Cont'd attempts collect debt not owed       36367
Account opening, closing, or management     23568
Disclosure verification of debt             16235
Communication tactics                       15312
Deposits and withdrawals                    14721
Application, originator, mortgage broker    11201
Billing disputes                             9600
Name: Issue, dtype: int64

In [20]:
for val in m.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['Issue']==val,1,0)
    cd_test[varname]=np.where(cd_test['Issue']==val,1,0)
del cd_train['Issue']
del cd_test['Issue']

In [21]:
cd_train.head()

,Product,Sub-product,Sub-issue,Consumer complaint narrative,Company public response,State,Tags,Consumer consent provided?,Submitted via,Company response to consumer,...,Issue_Loan_modification_collection_foreclosure,Issue_Incorrect_information_on_credit_report,Issue_Loan_servicing__payments__escrow_account,Issue_Cont'd_attempts_collect_debt_not_owed,Issue_Account_opening__closing__or_management,Issue_Disclosure_verification_of_debt,Issue_Communication_tactics,Issue_Deposits_and_withdrawals,Issue_Application__originator__mortgage_broker,Issue_Billing_disputes
0,Credit card,NaN,NaN,NaN,NaN,MI,Older American,NaN,Web,Closed with explanation,...,0,0,0,0,0,0,0,0,0,0
1,Bank account or service,(CD) Certificate of deposit,NaN,NaN,NaN,PA,NaN,NaN,Referral,Closed,...,0,0,0,0,0,0,0,0,0,0
2,Credit reporting,NaN,Account status,NaN,NaN,CA,NaN,NaN,Referral,Closed with non-monetary relief,...,0,1,0,0,0,0,0,0,0,0
3,Credit card,NaN,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,GA,Older American,Consent provided,Web,Closed with monetary relief,...,0,0,0,0,0,0,0,0,0,0
4,Credit card,NaN,NaN,NaN,NaN,MA,NaN,NaN,Web,Closed with explanation,...,0,0,0,0,0,0,0,0,0,0


In [22]:
k=cd_train['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['State']==val,1,0)
    cd_test[varname]=np.where(cd_test['State']==val,1,0)
del cd_train['State']
del cd_test['State']

In [23]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_train[varname]=np.where(pd.isnull(cd_train[col]),1,0)
    cd_train.drop([col],1,inplace=True)
    cd_test[varname]=np.where(pd.isnull(cd_test[col]),1,0)
    cd_test.drop([col],1,inplace=True)

In [24]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_train[col],prefix=col,drop_first=True)
    cd_train=pd.concat([temp,cd_train],1)
    cd_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(cd_test[col],prefix=col,drop_first=True)
    cd_test=pd.concat([temp,cd_test],1)
    cd_test.drop([col],1,inplace=True)

In [25]:
cd_train.head(5)

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_NC,State_MI,State_AZ,State_WA,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
3,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


In [26]:
cd_train.describe() 

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_NC,State_MI,State_AZ,State_WA,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
count,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,...,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000,478421.000000
mean,0.982977,0.740582,0.068820,0.128529,0.008745,0.029566,0.000002,0.014698,0.071939,0.065733,...,0.027898,0.025871,0.022346,0.020952,0.289438,0.611647,0.843038,0.811062,0.859525,0.716804
std,0.129356,0.438316,0.253148,0.334678,0.093107,0.169387,0.001446,0.120343,0.258387,0.247815,...,0.164681,0.158749,0.147808,0.143225,0.453502,0.487376,0.363765,0.391460,0.347479,0.450551
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
cd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 57 columns):
Timely response?_Yes                                            478421 non-null uint8
Company response to consumer_Closed with explanation            478421 non-null uint8
Company response to consumer_Closed with monetary relief        478421 non-null uint8
Company response to consumer_Closed with non-monetary relief    478421 non-null uint8
Company response to consumer_Closed with relief                 478421 non-null uint8
Company response to consumer_Closed without relief              478421 non-null uint8
Company response to consumer_Untimely response                  478421 non-null uint8
Submitted via_Fax                                               478421 non-null uint8
Submitted via_Phone                                             478421 non-null uint8
Submitted via_Postal mail                                       478421 non-null uint8
Submitted via_Referral       

In [28]:
cd_train.isnull().sum()

Timely response?_Yes                                            0
Company response to consumer_Closed with explanation            0
Company response to consumer_Closed with monetary relief        0
Company response to consumer_Closed with non-monetary relief    0
Company response to consumer_Closed with relief                 0
Company response to consumer_Closed without relief              0
Company response to consumer_Untimely response                  0
Submitted via_Fax                                               0
Submitted via_Phone                                             0
Submitted via_Postal mail                                       0
Submitted via_Referral                                          0
Submitted via_Web                                               0
Product_Consumer Loan                                           0
Product_Credit card                                             0
Product_Credit reporting                                        0
Product_De

In [29]:
cd_test.isnull().sum()

Timely response?_Yes                                            0
Company response to consumer_Closed with explanation            0
Company response to consumer_Closed with monetary relief        0
Company response to consumer_Closed with non-monetary relief    0
Company response to consumer_Closed with relief                 0
Company response to consumer_Closed without relief              0
Company response to consumer_Untimely response                  0
Submitted via_Fax                                               0
Submitted via_Phone                                             0
Submitted via_Postal mail                                       0
Submitted via_Referral                                          0
Submitted via_Web                                               0
Product_Consumer Loan                                           0
Product_Credit card                                             0
Product_Credit reporting                                        0
Product_De

In [30]:
cd_train['Timely response?_Yes'].value_counts()

1    470277
0      8144
Name: Timely response?_Yes, dtype: int64

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [33]:
x_train=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y_train=cd_train['Consumer disputed?']

In [34]:
x_train 

,Timely response?_Yes,Company response to consumer_Closed with explanation,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with relief,Company response to consumer_Closed without relief,Company response to consumer_Untimely response,Submitted via_Fax,Submitted via_Phone,Submitted via_Postal mail,...,State_NC,State_MI,State_AZ,State_WA,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
3,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478416,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,1,1,1
478417,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,1,1,1,1
478418,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,1,1
478419,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [35]:
y_train.value_counts()

0    376990
1    101431
Name: Consumer disputed?, dtype: int64

In [36]:
clf=LogisticRegression()


In [38]:
clf.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [40]:
prediction

array(['Yes', 'No', 'No', ..., 'No', 'No', 'No'], dtype='<U3')

In [41]:
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [42]:
submission

,Complaint ID,Consumer disputed?
0,675956,Yes
1,1858795,No
2,32637,No
3,1731374,No
4,501487,No
...,...,...
119601,153482,No
119602,2023523,No
119603,1413678,No
119604,732458,No


In [43]:
submission.to_csv('sample_submission.csv',index=False)

In [44]:
from sklearn.svm import SVR

In [45]:
svr_reg=SVR(kernel='rbf')

In [ ]:
svr_reg.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
 from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=5,scoring="roc_auc")

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
report(grid_search.cv_results_,5)


In [ ]:
logr.fit(x_train,y_train) 

In [ ]:
train_score=logr.predict_proba(x_train)[:,1] 

In [ ]:
train_score


In [ ]:
test_score=logr.predict_proba(cd_test)[:,1]


In [ ]:
test_score


In [ ]:
pd.DataFrame(test_score).to_csv("mysubmission_1.csv",index=False)
